In [1]:
import os
from models.midi_transformer import MIDITransformer

In [2]:
BASE_DIR = "/home/richhiey/Desktop/workspace/projects/virtual_musicians"
DATA_DIR = os.path.join(BASE_DIR, 'data', 'POP909-Dataset')
MODEL_SAVE_PATH = os.path.join(BASE_DIR, 'checkpoints', 'model1')
MODEL_CONFIG_PATH = os.path.join(BASE_DIR, 'model-store', 'models', 'configs', 'default.json')

In [3]:
transformer = MIDITransformer(MODEL_CONFIG_PATH, MODEL_SAVE_PATH)

TypeError: in user code:

    /home/richhiey/Desktop/workspace/projects/virtual_musicians/model-store/models/helpers/blocks.py:163 call  *
        inputx_d = self.enc_layers[i](
    /home/richhiey/Desktop/workspace/projects/virtual_musicians/model-store/models/helpers/layers.py:542 call  *
        last_memory = self.memory(embeddings)
    /home/richhiey/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:925 __call__  **
        return self._functional_construction_call(inputs, args, kwargs,
    /home/richhiey/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:1098 _functional_construction_call
        self._maybe_build(inputs)
    /home/richhiey/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2643 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /home/richhiey/Desktop/workspace/projects/virtual_musicians/model-store/models/helpers/layers.py:161 build
        shape=(